In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')


In [2]:
import os
import json

# Data Wrangling
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import glob
import flammkuchen as fl
import shutil

import seaborn as sns
import matplotlib as mpl

from tqdm import tqdm

## Custom functions from helper file

In [13]:
def sort_ipsi_contra_arrays(array1, array2, ipsi_indicator):
    # Determine ipsilateral and contralateral
    """
    ipsi_fin = 0, left fin is ipsi, 
    ipsi_fin = 1, right fin is ipsi_fin,
    """
    ipsi_fin = np.full(array1.shape[0], np.nan)
    contra_fin = np.full(array2.shape[0], np.nan)

    for i, dir_value in enumerate(ipsi_indicator):
        if dir_value == 0:  # Left 
            ipsi_fin[i] = array1[i]
            contra_fin[i] = array2[i]
        elif dir_value == 1:  # Right
            ipsi_fin[i] = array2[i]
            contra_fin[i] = array1[i]
    return ipsi_fin, contra_fin


def get_ipsi_contra_col(df, col1, col2, ipsi_indicator_col):
    """
    Function to separate ipsilateral and contralateral fin durations.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    col1 (str): The name of the first duration column (e.g., 'l_fin_duration').
    col2 (str): The name of the second duration column (e.g., 'r_fin_duration').
    ipsi_indicator_col (str): The name of the column indicating which fin is ipsi (0 for col1, 1 for col2).

    Returns:
    np.ndarray: Array of ipsilateral fin durations.
    np.ndarray: Array of contralateral fin durations.
    """
    # Determine ipsilateral and contralateral durations
    ipsi_fin_col = np.where(df[ipsi_indicator_col] == 0, df[col1], df[col2])
    contra_fin_col = np.where(df[ipsi_indicator_col] == 0, df[col2], df[col1])
    
    return ipsi_fin_col, contra_fin_col

## Read in Data

In [29]:
# master_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Raw_Data')


# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\230307_visstim_2D") #rectangular arena # start from fish 1
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_round")
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_2") #rectangular arena
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\13052024_visstim_2D_round")
master_path = Path(r"\\portulab.synology.me\data\Kata\Data\14052024_visstim_2D_round")

fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths

[WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f0'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f1'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f2'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f3'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f4'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f5'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f6'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f7'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f8'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f9')]

In [30]:
fish= 3
fish_id =  fish_paths[fish].name
exp_name = Path(fish_paths[fish]).parts[-2]
# exp_name = 'testfish'
fish_id, exp_name


('240514_f3', '14052024_visstim_2D_round')

In [31]:
# out_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Processed_Data')
## Analysed for paper

# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\230307_visstim_2D_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_round_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_2_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\13052024_visstim_2D_round_")
out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\14052024_visstim_2D_round_")

save_data_path = out_path

## Loop

In [32]:
cols = [
    ['l_fin_peak_vigor','r_fin_peak_vigor'],
    ['l_fin_freqs','r_fin_freqs'],
    ['l_osc', 'r_osc'],
    ['l_fin_duration', 'r_fin_duration'],
    ['l_max_amp', 'r_max_amp']
    # todo slopes add
]

In [33]:
for ind, fish_path in enumerate(tqdm(fish_paths)):
    fish_id =  fish_path.name
    print ('Working on fish {}'.format(fish_id))

    try:
        df = pd.read_csv(Path(out_path /'{}_features.csv'.format(fish_id)))  
        ipsi_fin = df.ipsi_fin
        ipsi_peak_vigor, contra_peak_vigor = get_ipsi_contra_col(df, 'l_fin_peak_vigor', 'r_fin_peak_vigor', 'ipsi_fin')
        ipsi_freqs, contra_freqs = get_ipsi_contra_col(df, 'l_fin_freqs', 'r_fin_freqs', 'ipsi_fin')
        ipsi_osc, contra_osc = get_ipsi_contra_col(df, 'l_osc', 'r_osc', 'ipsi_fin')
        ipsi_duration, contra_duration = get_ipsi_contra_col(df, 'l_fin_duration', 'r_fin_duration', 'ipsi_fin')
        ipsi_amp, contra_amp = get_ipsi_contra_col(df, 'l_max_amp', 'r_max_amp', 'ipsi_fin')

        df['ipsi_peak_vigor'] = ipsi_peak_vigor
        df['contra_peak_vigor'] = contra_peak_vigor
        df['ipsi_freqs'] = ipsi_freqs
        df['contra_freqs'] = contra_freqs
        df['ipsi_osc'] = ipsi_osc
        df['contra_osc'] = contra_osc
        df['ipsi_duration'] = ipsi_duration
        df['contra_duration'] = contra_duration
        df['ipsi_amp'] = ipsi_amp
        df['contra_amp'] = contra_amp

        df.to_csv(Path(out_path /'{}_features_ipsi_contra.csv'.format(fish_id)), index=False)  
    except:
        pass


 30%|████████████████████████▉                                                          | 3/10 [00:00<00:00, 27.03it/s]

Working on fish 240514_f0
Working on fish 240514_f1
Working on fish 240514_f2
Working on fish 240514_f3
Working on fish 240514_f4


 60%|█████████████████████████████████████████████████▊                                 | 6/10 [00:00<00:00, 24.51it/s]

Working on fish 240514_f5
Working on fish 240514_f6
Working on fish 240514_f7
Working on fish 240514_f8


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 19.88it/s]

Working on fish 240514_f9
